<a href="https://colab.research.google.com/github/aljuan14/Pemrosesan-Teks-Teori/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instal library yang diperlukan
!pip install nltk sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.5 MB/s eta 0:00:00


In [3]:
# --- Setup dan Load Data ---
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string

# Download resource NLTK (hanya perlu sekali)
nltk.download('punkt')
nltk.download('stopwords')

# Load dataset Anda
file_path = '//content/drive/MyDrive/Dataset analisis sentimen/analisis_sentimen_youtube_purbaya_vs_srimulyani (1).csv'
df = pd.read_csv(file_path)

# Pilih hanya kolom yang relevan
df_clean = df[['subjek', 'comment']].copy()

# Menghapus baris jika ada komentar yang kosong (meskipun data Anda 100% terisi)
df_clean = df_clean.dropna(subset=['comment'])
df_clean = df_clean.reset_index(drop=True)

print("Data Awal (sebelum diproses):")
print(df_clean.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Data Awal (sebelum diproses):
                 subjek                                            comment
0  Purbaya Yudhi Sadewa  Ga perlu seperti punyanya sahroni,ya pak. Semu...
1  Purbaya Yudhi Sadewa  Apa arti FAMILY OFFICE dan manfaatnya lngsung ...
2  Purbaya Yudhi Sadewa  MENTERI PURBAYA YUDHI SADEWA SATU TYPE DGN  MA...
3  Purbaya Yudhi Sadewa  Pak purbaya tidak memikirkan kekayaan dirinya ...
4  Purbaya Yudhi Sadewa  Pak Purbaya bukan orang yg suka hal2 mubazier....


In [4]:
# --- 1. Noise Removal (FUNGSI PERBAIKAN) ---

def remove_noise(text):
    # 1. Case Folding: Ubah semua teks menjadi lowercase
    text = text.lower()

    # 2. Hapus URL (http/https)
    text = re.sub(r'https?:\/\/\S+', '', text)

    # 3. Hapus @username (mentions)
    text = re.sub(r'@\w+', '', text)

    # 4. Hapus #hashtag
    text = re.sub(r'#\w+', '', text)

    # 5. Hapus angka
    text = re.sub(r'\d+', '', text)

    # 6. --- PERBAIKAN UTAMA ---
    #    Ganti tanda baca dengan SPASI, bukan menghapusnya
    #    Ini akan memisahkan "kosong,bisa" -> "kosong bisa"
    text = text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))

    # 7. --- TAMBAHAN ---
    #    Menormalkan karakter berulang (e.g., lhoooo -> lho)
    #    (Mengubah 3+ karakter berulang menjadi 1)
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # 8. Hapus emoji
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # 9. Hapus spasi berlebih (termasuk spasi dari penggantian tanda baca)
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)

    return text

# Terapkan ke kolom komentar
df_clean['cleaned_text'] = df_clean['comment'].apply(remove_noise)

print("\nData Setelah Noise Removal (Perbaikan):")
print(df_clean.head())


Data Setelah Noise Removal (Perbaikan):
                 subjek                                            comment  \
0  Purbaya Yudhi Sadewa  Ga perlu seperti punyanya sahroni,ya pak. Semu...   
1  Purbaya Yudhi Sadewa  Apa arti FAMILY OFFICE dan manfaatnya lngsung ...   
2  Purbaya Yudhi Sadewa  MENTERI PURBAYA YUDHI SADEWA SATU TYPE DGN  MA...   
3  Purbaya Yudhi Sadewa  Pak purbaya tidak memikirkan kekayaan dirinya ...   
4  Purbaya Yudhi Sadewa  Pak Purbaya bukan orang yg suka hal2 mubazier....   

                                        cleaned_text  
0  ga perlu seperti punyanya sahroni ya pak semua...  
1  apa arti family office dan manfaatnya lngsung ...  
2  menteri purbaya yudhi sadewa satu type dgn ma ...  
3  pak purbaya tidak memikirkan kekayaan dirinya ...  
4  pak purbaya bukan orang yg suka hal mubazier g...  


In [5]:
# --- 2. Normalization (Perbaikan Kamus) ---

# Menambahkan kata-kata baru
kamus_alay = {
    'yg': 'yang',
    'dgn': 'dengan',
    'utk': 'untuk',
    'gk': 'tidak',
    'gak': 'tidak',
    'ga': 'tidak',
    'utang': 'hutang',
    'byr': 'bayar',
    'pjk': 'pajak',
    'klo': 'kalau',
    'kalo': 'kalau',
    'smg': 'semoga',
    'moga': 'semoga',
    'jgn': 'jangan',
    'bgt': 'banget',
    'org': 'orang',
    'sbg': 'sebagai',
    'sdh': 'sudah',
    'blm': 'belum',
    'ttp': 'tetap',
    'tp': 'tapi',
    'krn': 'karena',
    'aja': 'saja',
    'menkeu': 'menteri keuangan',

    # --- TAMBAHAN BARU (dari temuan data) ---
    'sya': 'saya',
    'doank': 'saja',
    'twrlalu': 'terlalu',
    'lho': 'lho',
    'wkwk': '', # Anggap tawa tidak ada sentimen
    'wkwkwk': '',
    'kyk': 'seperti',
    'aamiin': 'amin', # Normalisasi
    'lngsung': 'langsung'
}

def normalize_text(text):
    words = text.split()
    normalized_words = [kamus_alay[word] if word in kamus_alay else word for word in words]
    return ' '.join(normalized_words)

# Terapkan ke kolom yang sudah dibersihkan
df_clean['normalized_text'] = df_clean['cleaned_text'].apply(normalize_text)

print("\nData Setelah Normalization (Perbaikan):")
print(df_clean.head())


Data Setelah Normalization (Perbaikan):
                 subjek                                            comment  \
0  Purbaya Yudhi Sadewa  Ga perlu seperti punyanya sahroni,ya pak. Semu...   
1  Purbaya Yudhi Sadewa  Apa arti FAMILY OFFICE dan manfaatnya lngsung ...   
2  Purbaya Yudhi Sadewa  MENTERI PURBAYA YUDHI SADEWA SATU TYPE DGN  MA...   
3  Purbaya Yudhi Sadewa  Pak purbaya tidak memikirkan kekayaan dirinya ...   
4  Purbaya Yudhi Sadewa  Pak Purbaya bukan orang yg suka hal2 mubazier....   

                                        cleaned_text  \
0  ga perlu seperti punyanya sahroni ya pak semua...   
1  apa arti family office dan manfaatnya lngsung ...   
2  menteri purbaya yudhi sadewa satu type dgn ma ...   
3  pak purbaya tidak memikirkan kekayaan dirinya ...   
4  pak purbaya bukan orang yg suka hal mubazier g...   

                                     normalized_text  
0  tidak perlu seperti punyanya sahroni ya pak se...  
1  apa arti family office dan manfaatnya la

In [6]:
# --- JALANKAN SEL INI UNTUK MEMPERBAIKI ERROR ---
import nltk
print("Mengunduh resource 'punkt_tab' yang diperlukan...")
nltk.download('punkt_tab')
print("Download 'punkt_tab' selesai.")

Mengunduh resource 'punkt_tab' yang diperlukan...


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Download 'punkt_tab' selesai.


In [7]:
# --- 3. Tokenization ---

def tokenize_text(text):
    return word_tokenize(text)

df_clean['tokenized_text'] = df_clean['normalized_text'].apply(tokenize_text)

print("\nData Setelah Tokenization:")
print(df_clean.head())


Data Setelah Tokenization:
                 subjek                                            comment  \
0  Purbaya Yudhi Sadewa  Ga perlu seperti punyanya sahroni,ya pak. Semu...   
1  Purbaya Yudhi Sadewa  Apa arti FAMILY OFFICE dan manfaatnya lngsung ...   
2  Purbaya Yudhi Sadewa  MENTERI PURBAYA YUDHI SADEWA SATU TYPE DGN  MA...   
3  Purbaya Yudhi Sadewa  Pak purbaya tidak memikirkan kekayaan dirinya ...   
4  Purbaya Yudhi Sadewa  Pak Purbaya bukan orang yg suka hal2 mubazier....   

                                        cleaned_text  \
0  ga perlu seperti punyanya sahroni ya pak semua...   
1  apa arti family office dan manfaatnya lngsung ...   
2  menteri purbaya yudhi sadewa satu type dgn ma ...   
3  pak purbaya tidak memikirkan kekayaan dirinya ...   
4  pak purbaya bukan orang yg suka hal mubazier g...   

                                     normalized_text  \
0  tidak perlu seperti punyanya sahroni ya pak se...   
1  apa arti family office dan manfaatnya langsung...  

In [8]:
# --- 4. Stopword Removal ---

list_stopwords = stopwords.words('indonesian')

custom_stopwords = [
    'purbaya', 'yudhi', 'sadewa', 'sri', 'mulyani', 'indrawati',
    'pak', 'bapak', 'bu', 'ibu', 'bpk', 'menteri', 'menkeu', 'keuangan',
    'kemenkeu', 'presiden', 'jokowi', 'prabowo', 'gibran',
    'amin' # Tambahkan 'amin' karena sudah dinormalisasi
]

list_stopwords.extend(custom_stopwords)
list_stopwords = set(list_stopwords)

def remove_stopwords(tokens):
    return [word for word in tokens if word not in list_stopwords]

df_clean['no_stopwords_text'] = df_clean['tokenized_text'].apply(remove_stopwords)

print("\nData Setelah Stopword Removal:")
print(df_clean.head())


Data Setelah Stopword Removal:
                 subjek                                            comment  \
0  Purbaya Yudhi Sadewa  Ga perlu seperti punyanya sahroni,ya pak. Semu...   
1  Purbaya Yudhi Sadewa  Apa arti FAMILY OFFICE dan manfaatnya lngsung ...   
2  Purbaya Yudhi Sadewa  MENTERI PURBAYA YUDHI SADEWA SATU TYPE DGN  MA...   
3  Purbaya Yudhi Sadewa  Pak purbaya tidak memikirkan kekayaan dirinya ...   
4  Purbaya Yudhi Sadewa  Pak Purbaya bukan orang yg suka hal2 mubazier....   

                                        cleaned_text  \
0  ga perlu seperti punyanya sahroni ya pak semua...   
1  apa arti family office dan manfaatnya lngsung ...   
2  menteri purbaya yudhi sadewa satu type dgn ma ...   
3  pak purbaya tidak memikirkan kekayaan dirinya ...   
4  pak purbaya bukan orang yg suka hal mubazier g...   

                                     normalized_text  \
0  tidak perlu seperti punyanya sahroni ya pak se...   
1  apa arti family office dan manfaatnya langsung.

In [9]:
# --- 5. Stemming (Sama, tetap akan LAMA) ---
# OPSI: Anda bisa lewati sel ini jika tidak ingin menunggu

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_tokens(tokens):
    return [stemmer.stem(word) for word in tokens]

print("\nMemulai proses stemming (mungkin butuh beberapa menit)...")
df_clean['stemmed_tokens'] = df_clean['no_stopwords_text'].apply(stem_tokens)
print("Proses stemming selesai.")

print("\nData Setelah Stemming:")
print(df_clean.head())


Memulai proses stemming (mungkin butuh beberapa menit)...
Proses stemming selesai.

Data Setelah Stemming:
                 subjek                                            comment  \
0  Purbaya Yudhi Sadewa  Ga perlu seperti punyanya sahroni,ya pak. Semu...   
1  Purbaya Yudhi Sadewa  Apa arti FAMILY OFFICE dan manfaatnya lngsung ...   
2  Purbaya Yudhi Sadewa  MENTERI PURBAYA YUDHI SADEWA SATU TYPE DGN  MA...   
3  Purbaya Yudhi Sadewa  Pak purbaya tidak memikirkan kekayaan dirinya ...   
4  Purbaya Yudhi Sadewa  Pak Purbaya bukan orang yg suka hal2 mubazier....   

                                        cleaned_text  \
0  ga perlu seperti punyanya sahroni ya pak semua...   
1  apa arti family office dan manfaatnya lngsung ...   
2  menteri purbaya yudhi sadewa satu type dgn ma ...   
3  pak purbaya tidak memikirkan kekayaan dirinya ...   
4  pak purbaya bukan orang yg suka hal mubazier g...   

                                     normalized_text  \
0  tidak perlu seperti punyany

In [11]:
# --- 6. Finalisasi (Gabung kembali token) ---

def join_tokens(tokens):
    return ' '.join(tokens)

# --- SESUAIKAN BARIS INI ---
# Jika Anda MENJALANKAN stemming:
df_clean['final_text'] = df_clean['stemmed_tokens'].apply(join_tokens)
#
# Jika Anda MELEWATI stemming (agar cepat):
# df_clean['final_text'] = df_clean['no_stopwords_text'].apply(join_tokens)

# Tampilkan hasil akhir
print("\nDataFrame Final Hasil Preprocessing (Perbaikan):")
final_df = df_clean[['subjek', 'final_text']]
print(final_df.head())

# Simpan hasil ke CSV baru
final_df.to_csv('analisis_sentimen_bersih.csv', index=False)
print("\nData bersih (v2) berhasil disimpan ke 'analisis_sentimen_cleaned_v2.csv'")


DataFrame Final Hasil Preprocessing (Perbaikan):
                 subjek                                         final_text
0  Purbaya Yudhi Sadewa  punya sahroni ya brandid trilyunan otak cek ko...
1  Purbaya Yudhi Sadewa         arti family office manfaat langsung rakyat
2  Purbaya Yudhi Sadewa  type ma rie muhammad cerdas sederhana manfaat ...
3  Purbaya Yudhi Sadewa  pikir kaya pikir rakyat yng mulya jalan amanah...
4  Purbaya Yudhi Sadewa  orang suka mubazier pk milyaran kl cm planga p...

Data bersih (v2) berhasil disimpan ke 'analisis_sentimen_cleaned_v2.csv'
